In [ ]:
from os import dup2
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

print("Method 1 or 2 ?\n1) recommended movies with (ideal & actual) ratings of test user, and calculate MSE.\n2) recommended movies with predicted ratings of test user.\n")
method = int(input())

Rating = pd.read_csv('Rating.txt', sep="\t")

Data = pd.read_csv('Data.txt', sep="\t").iloc[:,1:] # skip the first column

# Split the data 75% to train and 25% to test randomly
Train, Test = train_test_split(Data, test_size=0.25) 

Tree = KDTree(Train, metric='euclidean')

# Get the neighbors of the test user number 15
# you can work with another test user
nbr_neighbors = 3
Dst, Ind = Tree.query(Test.iloc[[15]], k=nbr_neighbors)

# Movies have neighbor ratings
neigbors_movies = Rating[ Rating['user'].isin(Train.iloc[Ind[0]].index+1) ]

# Movies have tstuser rating
tstusr_movies = Rating[ Rating['user'].isin(Train.iloc[[15]].index+1)]


if method == 1:
  # get the user movie names and his rating on them
  tstusr_ratings = tstusr_movies.iloc[:,2].reset_index(drop=True) # Method 1
  tstusr_movie_names = pd.read_csv('item.txt', sep="|",header=None).iloc[tstusr_movies.iloc[:,1]-1,1].reset_index(drop=True) # Method 1
  ideal_rating = pd.concat([tstusr_movie_names, tstusr_ratings], axis=1) # Method 1
  #ideal_rating.columns = ["recommended movies","ideal ratings"] # Method 1
  ideal_rating.columns = ["movie names","ideal ratings"] # Method 1


if method != 1:
  # The favorite movies of the neighbors without the common 
  # movies with the tstuser are the recommened item movies
  recommanded_item = np.setdiff1d(neigbors_movies.iloc[:,1],tstusr_movies.iloc[:,1]) # Method 2

# get the mean rating value for each common item of the neighbors
sum_ratings_of_each_item = neigbors_movies.groupby(by="item")["rating"].sum()
count_occurrence_of_each_item = neigbors_movies.pivot_table(columns=['item'], aggfunc='size')
predicted_ratings_of_recommended_items = sum_ratings_of_each_item.div(count_occurrence_of_each_item)

# Select the recommened movie names from the list of the movies
if method != 1:
  recommanded_movie_names = pd.read_csv('item.txt', sep="|",header=None).iloc[recommanded_item-1,1]  # Method 2
else:
  recommanded_movie_names = pd.read_csv('item.txt', sep="|",header=None).iloc[neigbors_movies.iloc[:,1].drop_duplicates()-1,1] # Method 1
# add 1 to all recommanded_movie_names indices in order to be equal
# to the predicted_ratings_of_recommended_items indices
recommanded_movie_names.index += 1

recommanded = pd.concat([recommanded_movie_names, predicted_ratings_of_recommended_items], axis=1).dropna()
recommanded.columns = ['recommended movies', 'predicted ratings']
if method != 1:
  print(recommanded.to_string(index=False))  # Method 2

if method == 1:
  ideal_and_actual_ratings = pd.merge(recommanded,ideal_rating, left_on = "recommended movies", right_on = "movie names") # Method 1
  ideal_and_actual_ratings.drop('movie names', axis=1, inplace=True) # Method 1
  print("MSE:",mean_squared_error(ideal_and_actual_ratings.iloc[:,1], ideal_and_actual_ratings.iloc[:,2], squared=True)) # Method 1
  print(ideal_and_actual_ratings.to_string(index=False)) # Method 1